In [1]:
from mobfot import MobFot
from datetime import date, timedelta
from tqdm import tqdm
from joblib import Parallel, delayed
import pandas as pd

In [2]:
def generar_fechas(anio_inicio, anio_fin):
  """
  Genera una lista de fechas en formato YYYYMMDD
  entre dos años (inclusive).

  Args:
    anio_inicio (int): Año de inicio (ej. 2017)
    anio_fin (int): Año de fin (ej. 2024)

  Returns:
    lista_fechas (list): Lista de strings con las fechas.
  """
  lista_fechas = []
  fecha_inicio = date(anio_inicio, 1, 1)
  fecha_fin = date(anio_fin, 12, 31)
  while fecha_inicio <= fecha_fin:
    fecha_str = fecha_inicio.strftime("%Y%m%d")
    lista_fechas.append(fecha_str)
    fecha_inicio += timedelta(days=1)
  return lista_fechas

# Ejemplo de uso
anio_inicio = 2015
anio_fin = 2023
lista_fechas = generar_fechas(anio_inicio, anio_fin)

In [3]:
client = MobFot()

In [4]:
def day_matches(date):
    client = MobFot()
    ids = []
    day_matches = client.get_matches_by_date(date)
    for leag in range(len(day_matches["leagues"])):
        for match_leg in range(len(day_matches["leagues"][leag]["matches"])):
            ids.append(day_matches["leagues"][leag]["matches"][match_leg]["id"])
    return ids

In [5]:
workers = 12
results = Parallel(n_jobs=workers)(delayed(day_matches)(d) for d in tqdm(lista_fechas))

100%|██████████| 3287/3287 [02:34<00:00, 21.24it/s]


In [6]:
ids = []
for r in results:
    ids.extend(r)
len(ids)

471436

In [8]:
def get_info(id):
    try : 
        client = MobFot()
        info = client.get_match_details(id)
        data = {
            "id": [id],
            'date': [info["general"]["matchTimeUTCDate"][:10]],
            'leagueName': [info["general"]["leagueName"]],
            'homeTeam': [info["general"]["homeTeam"]["name"]],
            'awayTeam': [info["general"]["awayTeam"]["name"]],
            'homeTeam_score': [info["header"]["teams"][0]["score"]],
            'awayTeam_score': [info["header"]["teams"][1]["score"]],
            'homeidTeam': [info["general"]["homeTeam"]["id"]],
            'awayidTeam': [info["general"]["awayTeam"]["id"]]
        }
        for i in range(len(info["content"]["stats"]["Periods"]["All"]["stats"])):
            for j in range(len(info["content"]["stats"]["Periods"]["All"]["stats"][i]["stats"])):
                key = info["content"]["stats"]["Periods"]["All"]["stats"][i]["stats"][j]["key"]
                data[key+"_home"] = info["content"]["stats"]["Periods"]["All"]["stats"][i]["stats"][j]["stats"][0]
                data[key+"_away"] = info["content"]["stats"]["Periods"]["All"]["stats"][i]["stats"][j]["stats"][1]
    
        df_id = pd.DataFrame(data)
        return df_id
    except Exception as error:
        print(error)
        return False

In [9]:
# La primera vez, hay que crear un data frame de 0
# df = pd.DataFrame()
# Después leer el archivo guardado
df = pd.read_csv('datos_fotmob_primer_filtro.csv')
workers = 10
trabajo_por_division = int(len(ids)/1000)
for div in tqdm(range(1, trabajo_por_division+1)):
    results = Parallel(n_jobs=workers)(delayed(get_info)(id)
                                       for id in ids[1000*(div-1):1000*div])
    for i in results:
        if str(type(i)) == "<class 'pandas.core.frame.DataFrame'>":
            df = pd.concat([df, i], ignore_index=True)
    df.to_csv('datos_fotmob_primer_filtro.csv', header=True, index=False)
    print(len(df))

  0%|          | 0/129 [00:00<?, ?it/s]

343


  1%|          | 1/129 [03:58<8:27:56, 238.10s/it]

89234
344


  2%|▏         | 2/129 [08:11<8:43:04, 247.12s/it]

...

140519
470


 99%|█████████▉| 128/129 [10:33:27<06:18, 378.39s/it]

140988
471


100%|██████████| 129/129 [10:40:32<00:00, 297.93s/it]

141468


In [13]:
df_sin_duplicados = df.drop_duplicates()
df_sin_duplicados = df_sin_duplicados.sort_values(by="date",ignore_index=True)
# Guardar la primera corrida de datos scrapeados
df_sin_duplicados.to_csv('datos_fotmob_primer_filtro.csv', header=True, index=False)

In [10]:
df_primer_filtro = pd.read_csv('datos_fotmob_primer_filtro.csv')
segundo_filtro = list(set(ids)-set(df_primer_filtro["id"]))

# La primera vez, hay que crear un data frame de 0
# df = pd.DataFrame()
# Después leer el archivo guardado
df_segudo_filtro = pd.read_csv('datos_fotmob_segundo_filtro.csv')
workers = 6
trabajo_por_division = int(len(segundo_filtro)/1000)
for div in tqdm(range(16, trabajo_por_division+1)):
    results = Parallel(n_jobs=workers)(delayed(get_info)(id)
                                       for id in segundo_filtro[1000*(div-1):1000*div])
    for i in results:
        if str(type(i)) == "<class 'pandas.core.frame.DataFrame'>":
            df_segudo_filtro = pd.concat([df, i], ignore_index=True)
    df_segudo_filtro.to_csv('datos_fotmob_segundo_filtro.csv', header=True, index=False)
    print(len(df_segudo_filtro))

  0%|          | 1/303 [01:42<8:37:38, 102.84s/it]

125


  1%|          | 2/303 [26:26<76:31:32, 915.26s/it]

125


  1%|          | 2/303 [29:08<73:05:45, 874.24s/it]


KeyboardInterrupt: 